In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
12,model_5_4_2,0.801591,0.482209,0.701225,0.555944,0.037118,0.103260,0.032320,0.069876,"Hidden Size=[30, 15], regularizer=0.02, learni..."
13,model_5_4_3,0.800774,0.459235,0.697457,0.539310,0.037271,0.107842,0.032727,0.072494,"Hidden Size=[30, 15], regularizer=0.02, learni..."
18,model_5_4_0,0.796070,0.807401,0.865006,0.827109,0.038151,0.038409,0.014603,0.027206,"Hidden Size=[30, 15], regularizer=0.02, learni..."
31,model_5_3_15,0.783443,0.776428,0.472276,0.717509,0.040513,0.041830,0.037718,0.039895,"Hidden Size=[30, 15], regularizer=0.02, learni..."
32,model_5_3_14,0.780775,0.775166,0.537533,0.732165,0.041012,0.042066,0.033054,0.037825,"Hidden Size=[30, 15], regularizer=0.02, learni..."
33,model_5_3_13,0.777595,0.774688,0.598539,0.746360,0.041607,0.042156,0.028694,0.035821,"Hidden Size=[30, 15], regularizer=0.02, learni..."
35,model_5_3_12,0.773531,0.773167,0.651757,0.757968,0.042367,0.042440,0.024890,0.034181,"Hidden Size=[30, 15], regularizer=0.02, learni..."
36,model_5_4_1,0.769240,0.729286,0.635777,0.700544,0.043170,0.053987,0.039399,0.047122,"Hidden Size=[30, 15], regularizer=0.02, learni..."
37,model_5_3_11,0.768582,0.770898,0.698319,0.767465,0.043293,0.042865,0.021562,0.032840,"Hidden Size=[30, 15], regularizer=0.02, learni..."
42,model_5_3_10,0.762082,0.760518,0.737223,0.769450,0.044509,0.044807,0.018782,0.032560,"Hidden Size=[30, 15], regularizer=0.02, learni..."
